# Regression mit Holdout und anderen Regressoren

In diesem Notebook wirst du mit dem Holdout-Verfahren und multidimensionaler Regression arbeiten. Außerdem wirst du dir anschauen, wie gut andere Regressoren funktionieren.

## Datenset laden

Dieses Vorgehen kennst du schon:

In [1]:
from sklearn import datasets
california = datasets.fetch_california_housing()

Wie gehabt wandelst du das Datenset in einen `DataFrame`.

In [2]:
import pandas as pd
df = pd.DataFrame(california.data, columns=california.feature_names)
df["Price"] = california.target

## Lineare Regression mit Trainings- und Test-Daten

Du kennst das *Holdout-Verfahren* schon von der Klassifikation. Hier kannst du es genauso anwenden:

In [3]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(df[["MedInc"]].values, df["Price"].values)

Die Regression `fit`test du nur mit den Trainingsdaten

In [4]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## Fehler berechnen

In diesem Fall reicht es, wenn du einen Score berechnest. Nutze den `r2`-Score:

In [5]:
from sklearn import metrics
metrics.r2_score(y_train, lr.predict(X_train))

0.4654059862740644

Wie sieht der Score bei den Testdaten aus?

In [6]:
metrics.r2_score(y_test, lr.predict(X_test))

0.49692791370025735

Sogar besser - von *Overfitting* kann hier also keine Rede sein!

## Mehrdimensionale Regression

Mit `sklearn` kannst du auch mehrdimensionale (oder sog. *multivariate*) Regression durchführen. Es ist gar nicht einfach, die richtigen Features zu finden. In diesem Beispiel benutzt du einfach *alle*:

In [7]:
(X_train, X_test, y_train, y_test) = train_test_split(california.data, california.target)

Der Aufruf unterscheidet sich nicht:

In [8]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

LinearRegression()

Überprüfe den Score:

In [9]:
metrics.r2_score(y_train, mlr.predict(X_train))

0.6054386876777129

Das ist viel besser als mit einem Feature! Allerdings liegt das zum Teil auch daran, dass der `r2`-Wert immer besser wird, wenn du mehr Features verwendest. Um die Qualität zu beurteilen, benötigst du den *justierten* `r2`:

In [10]:
X_train.shape

(15480, 8)

In [11]:
def adjusted_r2(r2, n, k):
    return 1 - ((1-r2)*(n-1)/(n-k-1))

adjusted_r2(metrics.r2_score(y_train, mlr.predict(X_train)), *X_train.shape)

0.6052346614028388

Da du nicht viele Features verwendest im Vergleich zu der Anzahl der Samples, ist der Unterschied sehr gering.

Wie sieht es beim Testdatenset aus?

In [12]:
adjusted_r2(metrics.r2_score(y_test, mlr.predict(X_test)), *X_test.shape)

0.6077232698644578

Auch besser! Das sieht auch hier nicht nach Overfitting aus.

Wenn du möchtest, kannst du `sklearn` ausrechnen lassen, welche Features die wichtigsten sind. Dazu dient die Funktion `permutation_importance':

In [13]:
from sklearn import inspection
pi = inspection.permutation_importance(mlr, X_train, y_train)
pi

{'importances_mean': array([1.03039242e+00, 2.04317660e-02, 1.05028979e-01, 1.39202692e-01,
        2.52097979e-05, 1.23392098e-03, 1.23227041e+00, 1.15795720e+00]),
 'importances_std': array([1.00745408e-02, 1.12484020e-03, 2.76231270e-03, 3.34777829e-03,
        3.44040126e-05, 2.28079292e-04, 1.02283640e-02, 8.70502298e-03]),
 'importances': array([[ 1.02527872e+00,  1.02376730e+00,  1.02039688e+00,
          1.03414139e+00,  1.04837781e+00],
        [ 2.01405324e-02,  1.87700910e-02,  2.07128302e-02,
          2.02667344e-02,  2.22686421e-02],
        [ 1.08515663e-01,  1.02237237e-01,  1.04357453e-01,
          1.02038191e-01,  1.07996353e-01],
        [ 1.35508288e-01,  1.42864575e-01,  1.41886265e-01,
          1.40899126e-01,  1.34855205e-01],
        [-7.40981813e-06,  2.29766899e-05,  1.60829181e-05,
          9.07814285e-05,  3.61777137e-06],
        [ 1.47757258e-03,  8.11175840e-04,  1.37030263e-03,
          1.29069694e-03,  1.21985689e-03],
        [ 1.21890412e+00,  1.2

Etwas übersichtlicher kannst du das in einem `DataFrame` darstellen:

In [14]:
idf = pd.DataFrame()
idf["feature"] = california.feature_names
idf["importance"] = pi.importances_mean
idf

,feature,importance
0,MedInc,1.030392
1,HouseAge,0.020432
2,AveRooms,0.105029
3,AveBedrms,0.139203
4,Population,0.000025
5,AveOccup,0.001234
6,Latitude,1.232270
7,Longitude,1.157957


In diesem Fall sind `Latitude` und `Longitude` sogar noch wichtigere Features. Bei einem genügend großen Datenset definieren diese die Lage der Objekte, die ganz offensichtlich eine Auswirkung auf den Preis hat.

## Unterschiedliche Regressoren

Neben der linearen Regression gibt es noch andere Regressoren, die du ausprobieren kannst. Du lässt dir immer den `r2` für Trainings- und Testdaten ausgeben, um den Regressor beurteilen zu können.

In [15]:
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.ensemble import ExtraTreesRegressor
for r in [LinearRegression(), SGDRegressor(loss='squared_error', penalty='l2',  random_state=42), 
          Ridge(), ExtraTreesRegressor(n_estimators=10,random_state=42)]:
    print(r)
    r.fit(X_train, y_train)
    print(metrics.r2_score(y_train, r.predict(X_train)), 
          metrics.r2_score(y_test, r.predict(X_test)))
          

LinearRegression()
0.6054386876777129 0.6083315687287889
SGDRegressor(random_state=42)
-3.394145021496717e+29 -3.4509337964475206e+29
Ridge()
0.6054386448675063 0.608326051036407
ExtraTreesRegressor(n_estimators=10, random_state=42)
1.0 0.7989843690654626


`ExtraTreesRegressor` ist eine *Mischung* verschiedener Ensemble-Verfahren. Obwohl die Traingsdaten überbestimmt scheinen, ist die Performance auch in den Testdaten sehr gut.

Der `ExtraTreesRegressor` kann direkt die wichtigsten Features ausgeben:

In [16]:
idf = pd.DataFrame()
idf["feature"] = california.feature_names
idf["importance"] = r.feature_importances_
idf

,feature,importance
0,MedInc,0.486635
1,HouseAge,0.068965
2,AveRooms,0.053348
3,AveBedrms,0.034788
4,Population,0.027866
5,AveOccup,0.109652
6,Latitude,0.105081
7,Longitude,0.113665


Auch hier siehst du, dass neben dem Einkommen auch die Lage eine Rolle spielt.

## Overfitting bei Regression

Wie bei Klassifikatoren gibt es auch bei Regressoren *Overfitting*. Um das beurteilen zu können, musst du die Daten in ein Trainings- und Testdatenset splitten.

Im Gegensatz zur Klassifikation kannst du hier mit einem Score arbeiten, was die Beurteilung insgesamt einfacher macht.